In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [ ]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [ ]:
transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

In [ ]:
transactions_train['max_purc_date'] = transactions_train.groupby('customer_id')['t_dat_datetime'].transform('max')

transactions_train['min_purc_date'] = transactions_train.groupby('customer_id')['t_dat_datetime'].transform('min')

transactions_train['total_visits'] = transactions_train.groupby('customer_id')['t_dat'].transform('nunique')

transactions_train['total_item_bought'] = transactions_train.groupby('customer_id')['article_id'].transform('nunique')

transactions_train['max_year'] = transactions_train.groupby('customer_id')['t_year'].transform('max')

In [ ]:
# transactions_train_filt = transactions_train[transactions_train['max_year'] == 2020]

# transactions_train_filt['product_reach'] = transactions_train_filt.groupby('article_id')['customer_id'].transform('nunique')

# transactions_train_filt = transactions_train_filt[transactions_train_filt['product_reach'] > 8]

transactions_train['repeat_purchase'] = transactions_train.groupby(['customer_id','article_id'])['t_dat'].transform('nunique')

transactions_train_filt = transactions_train[transactions_train['repeat_purchase'] > 1]

# transactions_train_filt = transactions_train_filt[transactions_train_filt['total_visits'] >= 2]

# transactions_train_filt = transactions_train_filt[transactions_train_filt['total_item_bought'] >= 12]

In [ ]:
# article_cust = transactions_train_filt[['article_id','customer_id']].drop_duplicates()

# article_cust_full = pd.merge(article_cust, article_cust.rename(columns={'article_id':'article_id_1',
#                                                    }),
#          left_on='customer_id',
#          right_on='customer_id'
#         )

# article_cust_full = article_cust_full[article_cust_full['article_id'] != article_cust_full['article_id_1']]

# common_cust = article_cust_full.groupby(['article_id', 'article_id_1']).\
# agg(common_cust=('customer_id','nunique')).reset_index()

# common_cust.shape

# common_cust = pd.merge(common_cust,
#                        article_cust_full.groupby('article_id').\
#                        agg(article_id_cus=('customer_id','nunique')).\
#                        reset_index(),
#                        left_on = 'article_id',
#                        right_on = 'article_id'
#                       )
# common_cust.head()

# common_cust.shape

# common_cust = pd.merge(common_cust,
#                        article_cust_full.groupby('article_id_1').\
#                        agg(article_id_cus=('customer_id','nunique')).\
#                        reset_index(),
#                        left_on = 'article_id_1',
#                        right_on = 'article_id_1'
#                       )
# common_cust.head()

# common_cust['lift'] = common_cust['common_cust']/ (common_cust['article_id_cus_x']*common_cust['article_id_cus_y'])

# common_cust = common_cust[common_cust['common_cust']>10].sort_values(['article_id_cus_y','lift'], ascending=[False,False])

# len(common_cust['article_id'].unique())

# common_cust = pd.merge(common_cust,
#          transactions_train_filt.groupby(['article_id']).\
#          agg(article_id_max_dat=('t_dat_datetime','max')).\
#          reset_index(),
#          left_on='article_id',
#          right_on='article_id'
#         )

# common_cust = pd.merge(common_cust,
#          transactions_train_filt.rename(columns={}).groupby(['article_id']).\
#          agg(article_id_1_max_dat=('t_dat_datetime','max')).\
#          reset_index(),
#          left_on='article_id_1',
#          right_on='article_id'
#         )

# common_cust.drop(['article_id_y'], axis = 1, inplace=True)
# common_cust.head()

# len(transactions_train_filt[(transactions_train_filt['t_month'].isin([8,9,10]))&
#                             (transactions_train_filt['t_year']!=2020)
                            
#                            ]['customer_id'].unique())



In [ ]:
product_purchase_cycle = transactions_train_filt[['customer_id','article_id','t_dat_datetime']].\
sort_values(['customer_id','article_id','t_dat_datetime']).drop_duplicates()

product_purchase_cycle['t_dat_datetime_lag'] = product_purchase_cycle.\
groupby(['customer_id','article_id'])['t_dat_datetime'].shift(1)

product_purchase_cycle['purchase_cycle'] = product_purchase_cycle['t_dat_datetime'] - product_purchase_cycle['t_dat_datetime_lag']

Average_Purchase_Cycle = product_purchase_cycle[~(product_purchase_cycle['purchase_cycle'].isnull())].\
groupby(['customer_id','article_id']).agg({"purchase_cycle":"mean"}).reset_index()

transactions_train_filt = pd.merge(transactions_train_filt,
                                   Average_Purchase_Cycle,
                                   left_on = ['customer_id','article_id'],
                                   right_on = ['customer_id','article_id'],
                                   how='inner'
                                  )

transactions_train_filt['product_last_purchase_date'] = transactions_train_filt.\
groupby(['customer_id','article_id'])['t_dat_datetime'].transform("max")

In [ ]:
from datetime import datetime
from datetime import timedelta

transactions_train_filt['purchase_cycle_days'] = transactions_train_filt['purchase_cycle'].astype('str').str.split(' ', expand=True)[0]

transactions_train_filt['next_purchase_date'] = transactions_train_filt['product_last_purchase_date'] + transactions_train_filt['purchase_cycle_days'].astype(int).map(timedelta)

In [ ]:
start_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(-90)
end_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(90)

In [ ]:
datetime.date(transactions_train_filt['t_dat_datetime'].max()) 

In [ ]:
start_date, end_date

In [ ]:
len(transactions_train_filt['customer_id'].unique())

In [ ]:
customer_next_purchase = transactions_train_filt[transactions_train_filt['next_purchase_date'].\
                        between(start_date, end_date, inclusive="both")]\
[['customer_id','t_dat','article_id']].drop_duplicates()

customer_next_purchase['repeat_purchase'] = customer_next_purchase.groupby(['customer_id'])['article_id'].transform('count')

customer_next_purchase['product_imp_rank'] = customer_next_purchase.groupby(['customer_id'])['repeat_purchase'].rank(method="first",ascending=True)

customer_next_purchase['max_rank'] = customer_next_purchase[customer_next_purchase['product_imp_rank'] <= 12].\
groupby('customer_id')['product_imp_rank'].transform('max')

customer_next_purchase = customer_next_purchase[['customer_id','article_id','max_rank']]

customer_next_purchase = customer_next_purchase.sort_values(['customer_id','article_id'])

In [ ]:
len(customer_next_purchase['customer_id'].unique())

In [ ]:
# Filling Products 

top_12_products = customer_next_purchase.groupby(['article_id']).agg(cust_count=('customer_id','nunique')).reset_index()

top_12_products = top_12_products.sort_values('cust_count', ascending = False).head(12)[['article_id']]

In [ ]:
filler_cust_prod_comb= pd.merge(customer_next_purchase['customer_id'].drop_duplicates(), 
top_12_products['article_id'], how = 'cross')

In [ ]:
customer_next_purchase= pd.concat([customer_next_purchase[['customer_id', 'article_id']],
          filler_cust_prod_comb[['customer_id', 'article_id']]],
          axis = 0
         ).drop_duplicates()

In [ ]:
customer_next_purchase['row_num'] = customer_next_purchase.groupby(['customer_id']).cumcount()

customer_next_purchase = customer_next_purchase[customer_next_purchase['row_num'] <= 11]

customer_next_purchase['article_id'] = ' 0'+customer_next_purchase['article_id'].astype('str')

customer_next_purchase = customer_next_purchase.groupby(['customer_id'])['article_id'].sum().reset_index()

In [ ]:
top_12_products['article_id'] = ' 0'+top_12_products['article_id'].astype('str')

top_12_products_for_missin_cust = top_12_products['article_id'].sum()
top_12_products_for_missin_cust

In [ ]:
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')\
[['customer_id']]

sample_submission = pd.merge(sample_submission,
                             customer_next_purchase,
                             left_on = ['customer_id'],
                             right_on = ['customer_id'],
                             how = 'left'
                            ).fillna(top_12_products_for_missin_cust)

In [ ]:
# no_products_cust['article_id'].fillna( "0706016001 0706016002 0372860001 0399223001 0464297007 0160442007 0610776002 0156231001 0399256001 0372860002 0706016003 0759871002", inplace=True)
# no_products_cust.shape

In [ ]:
sample_submission.rename(columns={'article_id':'prediction'}, inplace=True)
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index =False)